# Topological clustering algorithm

In [3]:
import tensorflow as tf
print(f"Tensorflow version : {tf.__version__}")

Tensorflow version : 2.2.0-rc1


In [4]:
# Moliere radius for lead (Wigmans2017, Appendix B)
moliereRadius = 16.0 / 3 # moliere radius in units of smallest irreducible unit

In [5]:
TSEED = tf.constant([1000.], tf.float32)
TNEIGHBOR = tf.constant([100.], tf.float32)
TCELL = tf.constant([50.], tf.float32)
TENERGY = tf.constant([15e3], tf.float32)

Groups neighboring cells of significant energy into clusters with a variable number of cells (vs sliding-window algorithm). Cluster growth starts at seed cells, i.e. cells with significance above $t_\text{seed}$ (high). Neighboring cells become new seeds if their significance are above $t_\text{neighbor}$ (medium). If a neighboring cell has significance only above $t_\text{cell}$ (low) it is simply added to the cluster without functioning as a seed. The low threshold means shower tails are not discarded, the other thresholds supress noise. This algorithm consists of two steps: the cluster maker and the cluster splitter.

## Cluster maker

Forms topological clusters from cells and consists of the following steps:

* **Finding seeds**: seed list consists of cells with significance above $t_\text{seed}$. Neighboring seed cells form initial proto-clusters (vs paper where each seed cell forms a proto-cluster) - possibly look at later. NB missing noise!
* **Finding neighbors**: current seed list ordered in descending order in significance. Consider neighboring cells of each seed cell in turn: a) if significance above $t_\text{neighbor}$ it is added to the neighbor seed list and adjacent proto-cluster; if adjacent to multiple proto-clusters, these are merged. b) if significance only above $t_\text{cell}$ cell included only in first adjacent proto-cluster (which has the more significant neighbor due to the sorting), not in the neighbor seed list (it is unclear from the paper if the latter is true). When all seed cells have been processed, the neighbor seed list becomes the new seed list. This procedure of neighbor finding is repeated until the seed list is empty.
* **Finalize**: resulting proto-clusters are sorted in descending order in $E_T$ and filtered $E_T>t_\text{energy}$ - does $E_T$ refer to transverse energy or total energy or total energy above threshold?  

"Neighboring" is here defined as the eight surronding cells within the same channel and overlapping cells in adjacent channels. The scintillating and Cerenkov signals are treated as two different channels, consequently, each cell has nine neighbors. 

### Finding seeds

In [6]:
from skimage.measure import label

@tf.function
def finding_seeds(image):
    """Finding seeds and create seed list"""
    seed_mask = tf.math.greater(image, TSEED)
    f = lambda x: label(x, connectivity=2)
    labels = tf.numpy_function(f, [seed_mask], Tout=tf.int64)
    seed_idx = tf.where(tf.not_equal(labels, 0))
    seed_id = tf.expand_dims(tf.gather_nd(labels, seed_idx), axis=1)
    seed_list = tf.concat([seed_id, seed_idx], axis=-1)
    return seed_list

### Finding neighbors

In [7]:
@tf.function
def sorting_seedlist(s, m):
    """Sorts current seed list in descending order"""
    sid, sidx = tf.split(s, num_or_size_splits=[1,3], axis=1)
    seeds = tf.gather_nd(m, sidx)
    indices = tf.math.top_k(seeds, k=tf.shape(s)[0])[1]
    return tf.gather(s, indices)

In [8]:
@tf.function
def neighbors(m, index, excl=tf.zeros([0, 3], tf.int64), incl=tf.zeros([0, 3], tf.int64)):
    """8-1 connectivity"""
    dense_shape = tf.shape(m, out_type=tf.int64)
    
    # indices of 3x3x1 centred at (i,j,k)
    i, j, k = tf.unstack(index)
    ii, jj = tf.meshgrid(tf.range(i-1, i+2), tf.range(j-1, j+2), indexing='ij')
    ii = tf.reshape(ii, [-1, 1])
    jj = tf.reshape(jj, [-1, 1])
    kk = tf.repeat(tf.reshape(k, (1, 1)), repeats=[9], axis=0)
    a = tf.concat([ii, jj, kk], axis=-1)
    
    # append index (i,j) in other k channel
    a = tf.concat([a, [[i, j, tf.math.abs(k-1)]]], axis=0)
    
    # remove centre point (i,j,k)
    mask_centre = tf.reduce_any(tf.not_equal(a, index), axis=-1)
    
    # remove points outside image
    b = tf.expand_dims(a, axis=-1)
    height = tf.reshape(dense_shape[0], [-1,1])
    width = tf.reshape(dense_shape[1], [-1,1])
    negone = tf.constant([[-1]], dense_shape.dtype)
    height_bound = tf.concat([height, negone, negone], axis=1)
    width_bound = tf.concat([negone, width, negone], axis=1)
    c = tf.transpose(tf.concat([height_bound, width_bound], axis=0))
    mask_boundary = tf.reduce_all(tf.not_equal(b, c), axis=[-2,-1])
    mask = tf.logical_and(mask_centre, mask_boundary)
    a = tf.boolean_mask(a, mask)
    
    # remove points in excl and only consider points in incl
    a_sp = tf.SparseTensor(a, tf.ones(tf.shape(a)[0], dtype=tf.bool), dense_shape)
    a_d = tf.sparse.to_dense(tf.sparse.reorder(a_sp))
    excl_sp = tf.SparseTensor(excl, tf.ones(tf.shape(excl)[0], dtype=tf.bool), dense_shape)
    excl_d = tf.sparse.to_dense(tf.sparse.reorder(excl_sp))
    incl_sp = tf.SparseTensor(incl, tf.ones(tf.shape(incl)[0], dtype=tf.bool), dense_shape)
    incl_d = tf.sparse.to_dense(tf.sparse.reorder(incl_sp))
    mask_final = tf.logical_and(a_d, tf.logical_not(excl_d))
    
    true_fn = lambda: tf.logical_and(mask_final, incl_d)
    mask_final = tf.cond(tf.greater(tf.shape(incl)[0], 0), true_fn, lambda: mask_final)
    
    neighbors = tf.where(mask_final)
    return neighbors

In [18]:
@tf.function
def merge_adjacent_proto(nj, siid, m, p, s, l):
    """Merge adjacent proto-clusters"""
    sid, sidx = tf.split(s, num_or_size_splits=[1,3], axis=1)
    lid, lidx = tf.split(l, num_or_size_splits=[1,3], axis=1)
    proto_id, proto_idx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    nnj = neighbors(m, nj, excl=sidx)
    dense_shape=tf.shape(m, out_type=tf.int64)
    nnj_sp = tf.SparseTensor(nnj, tf.ones(tf.shape(nnj)[0], tf.bool), dense_shape)
    nnj_d = tf.sparse.to_dense(tf.sparse.reorder(nnj_sp))
    proto_sp = tf.SparseTensor(proto_idx, tf.reshape(proto_id, [-1,]), dense_shape)
    proto_d = tf.sparse.to_dense(tf.sparse.reorder(proto_sp))
    indices = tf.where(tf.logical_and(tf.logical_and(nnj_d, tf.not_equal(proto_d, 0)), tf.greater(m, TNEIGHBOR)))
#     indices = tf.where(tf.logical_and(nnj_d, tf.not_equal(proto_d, 0)))
    values = tf.gather_nd(proto_d, indices)
    values = tf.boolean_mask(values, tf.not_equal(values, siid))
    
    # loop over values
    k0 = tf.constant(0)
    ck = lambda k, p, s, l: tf.less(k, tf.shape(values)[0])
    def bk(k, p, s, l):
        neigh_id = tf.gather(values, k)
        pnew = tf.where(tf.equal(proto_d, neigh_id), tf.reshape(siid, [-1,])*tf.ones_like(p), p)
        snew = tf.where(tf.equal(sid, neigh_id), tf.reshape(siid, [-1,])*tf.ones_like(s), s)
        lnew = tf.where(tf.equal(lid, neigh_id), tf.reshape(siid, [-1,])*tf.ones_like(l), l)
        return [tf.add(k, 1), pnew, snew, lnew]
    k, p, sid, lid = tf.while_loop(ck, bk, loop_vars=[k0, proto_d, sid, lid],
                                   shape_invariants=[k0.get_shape(), proto_d.get_shape(),
                                                     sid.get_shape(), lid.get_shape()])                
    indices = tf.where(tf.not_equal(p, 0))
    values = tf.expand_dims(tf.gather_nd(p, indices), axis=1)
    pnew = tf.concat([values, indices], axis=1)
    snew = tf.concat([sid, sidx], axis=1)
    lnew = tf.concat([lid, lidx], axis=1)
    return [pnew, snew, lnew]

@tf.function
def bj_maker(j, m, p, s, l, n, siid):
    """Body of while loop over neighbors"""
    nj = tf.gather(n, j)
    nval = tf.gather_nd(m, nj)
    nnew = tf.expand_dims(tf.concat([siid, nj], axis=0), axis=0)

    def tneighbor(siid=siid):
        """Append cells to both proto-clusters and neighbor seed list if value above TNEIGHBOR"""
        [pnew, snew, lnew] = merge_adjacent_proto(nj, siid, m, p, s, l)
        pnew = tf.concat([pnew, nnew], axis=0)
        lnew = tf.concat([lnew, nnew], axis=0)
        return [pnew, snew, lnew]

    def tcell():
        """Append cells to proto-clusters if value above TCELL"""
        def true_fn():
            return tf.concat([p, nnew], axis=0)
        def false_fn(): return p
        pnew = tf.cond(tf.greater(nval, TCELL), true_fn, false_fn)
        return [pnew, s, l]
    
    [pnew, snew, lnew] = tf.cond(tf.greater(nval, TNEIGHBOR), true_fn=tneighbor, false_fn=tcell)
    
    return [tf.add(j, 1), m, pnew, snew, lnew, n, siid]
    
@tf.function
def bi_maker(i, m, p, s, l):
    """Body of while loop over current seed list"""
    si = tf.gather(s, i)
    
    # find neighbours not in protolist
    siid, siidx = tf.split(si, num_or_size_splits=[1,3], axis=0)
    _, pidx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    n = neighbors(m, siidx, excl=pidx)
        
    # loop over neighbors
    j0 = tf.constant(0)
    cj = lambda j, m, p, s, l, n, siid: tf.less(j, tf.shape(n)[0])
    _, m, p, s, l, _, _ = tf.while_loop(
        cj, bj_maker, loop_vars=[j0, m, p, s, l, n, siid],
        shape_invariants=[j0.get_shape(), m.get_shape(), tf.TensorShape([None, 4]),
                          s.get_shape(), tf.TensorShape([None, 4]),
                          n.get_shape(), siid.get_shape()])
    
    return [tf.add(i, 1), m, p, s, l]

@tf.function
def finding_neighbors_maker(m, p, s):    
    # sort current seed list in descending order
    ssort = sorting_seedlist(s, m)
    
    # loop over current seed list
    i0 = tf.constant(0)
    l0 = tf.zeros([0,4], tf.int64)
    ci = lambda i, m, p, s, l: tf.less(i, tf.shape(ssort)[0])
    _, m, pnew, s, lnew = tf.while_loop(
        ci, bi_maker, loop_vars=[i0, m, p, s, l0],
        shape_invariants=[i0.get_shape(), m.get_shape(), tf.TensorShape([None, 4]),
                          s.get_shape(), tf.TensorShape([None, 4])])
    
    # neighbor seed list becomes the new seed list
    return pnew, lnew

### Finalize cluster maker

In [19]:
@tf.function
def finalize_maker(m, p):
    """Filter clusters by energy threshold"""
    # convert to dense tensor
    pid, pidx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    dense_shape=tf.shape(m, out_type=tf.int64)
    psp = tf.SparseTensor(pidx, tf.reshape(pid, [-1,]), dense_shape)
    pd = tf.sparse.to_dense(tf.sparse.reorder(psp))

    # loop over unique proto-cluster id's
    u = tf.unique(tf.reshape(pid, [-1]), out_idx=pid.dtype)[0]
    i0 = tf.constant(0)
    c = lambda i, pdi: tf.less(i, tf.size(u))
    def b(i, pdi):
        ui = tf.gather(u, i)
        mask = tf.equal(pdi, ui)
        indices = tf.where(mask)
        values = tf.gather_nd(m, indices)
        Ei = tf.reduce_sum(values)
        condition = tf.logical_and(mask, tf.less(Ei, TENERGY))
        pdi = tf.where(condition, tf.zeros_like(pdi), pdi)
        return [tf.add(i, 1), pdi]
    pdnew = tf.while_loop(c, b, loop_vars=[i0, pd],
                          shape_invariants=[i0.get_shape(), pd.shape])[1]
    
    # convert to proto-list
    pidxnew = tf.where(pdnew)
    pidnew = tf.reshape(tf.gather_nd(pdnew, pidxnew), [-1,1])
    pnew = tf.concat([pidnew, pidxnew], axis=1)
    return pnew

### Cluster maker

In [20]:
@tf.function
def cluster_maker(parsed):
    """m: image, p: protolist, s: seedlist, l: neighlist, n: neighbor"""
#     global tseed
#     tseed = parsed['tseed']
    
    s = finding_seeds(parsed['image'])
    
    # finding neighbours recursively until current seed list is empty
    c = lambda pi, si: tf.not_equal(tf.size(si), 0)
    b = lambda pi, si: finding_neighbors_maker(parsed['image'], pi, si)
    pnew = tf.while_loop(c, b, loop_vars=[s, s],
                         shape_invariants=[tf.TensorShape([None, 4]), tf.TensorShape([None, 4])])[0]
    
    parsed = parsed.copy()
    parsed['proto'] = finalize_maker(parsed['image'], pnew)
    return parsed

## Cluster splitter

The cluster maker is sufficient for isolated signals, but not for overlapping showers. If individual particles form local maxima they may still be separable. Acting on the clusters resulting from the cluster maker, this is what the cluster splitter does in the followning steps:

* **Finding local maxima**: a local maxima is defined as a cell with: a) $E>t_\text{locmax}$, b) energy greater than that of any neighboring cell, and c) number of neighboring cells withing the parent cluster $N>t_\text{number}$ (default is $\geq4$). Each local maximum forms a cluster and parent clusters without any local maximum cell will not be split.
* **Finding neighbors**: the local maxima now becomes the initial seed list much like in cluster maker, except that only cells originally clustered are used, without thresholding and merging. Instead of merging, shared cells are added to a shared cell list to be handled separately.
* **Shared cells**: the shared cell list is expanded iteratively adding neighbors from the originally clustered cells not yet assigned to any proto-cluster. Each of these are then added to the two adjoining proto-clusters with the weights $w_1=\frac{E_{1}}{E_{1}+rE_{2}}, w_{2}=1-w_{1}, r=\exp(d_{1}-d_{2})$, where $E_{1,2}$ are the energies of the two proto-clusters and $d_{1,2}$ are the distances of the shared cell to the proto-cluster centroids in units of a typical em shower scale.
* **Finalize**:

### Finding local maxima

In [14]:
def finding_local_maxima(m, p):
    """Find local maxima cells by looping over protolist.
    m: filtered image, x: local maxima"""
    
    # loop over protolist
    i0 = tf.constant(0)
    x = tf.zeros([0, 4], p.dtype)
    c = lambda i, p, x: tf.less(i, tf.shape(p)[0])
    def b(i, p, x):
        pi = tf.gather(p, i)
        piid, piidx = tf.split(pi, num_or_size_splits=[1,3], axis=0)
        pval = tf.gather_nd(m, piidx)
        midx = tf.where(tf.greater(m, 0))
        nidx = neighbors(piidx, incl=midx)
        m_val = tf.gather_nd(m, nidx)
#         m_val = tf.boolean_mask(m_val, tf.greater(m_val, 0))

        pred = tf.logical_and(tf.logical_and(tf.greater(pval, TLOCMAX), 
                                      tf.greater(pval, tf.math.reduce_max(m_val))),
                                      tf.greater(m_val.get_shape()[0], TNUMBER))
        r = tf.cond(pred, lambda: tf.expand_dims(pi, axis=0), lambda: tf.zeros([0, 4], pi.dtype))
        
        return [tf.add(i, 1), p, tf.concat([x, r], axis=0)]
    
    _, p, x = tf.while_loop(
        c, b, loop_vars=[i0, p, x], 
        shape_invariants=[i0.get_shape(), p.get_shape(), tf.TensorShape([None, 4])])
    
    return x

### Finding neighbors

In [13]:
def is_shared(nj, p, siid):
    nnj = neighbors(nj)
    proto_id, proto_idx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    nnj_sp = tf.SparseTensor(nnj, tf.ones(tf.shape(nnj)[0], tf.bool), dense_shape=[HEIGHT, WIDTH, 2])
    nnj_d = tf.sparse.to_dense(tf.sparse.reorder(nnj_sp))
    proto_sp = tf.SparseTensor(proto_idx, tf.reshape(proto_id, [-1,]), dense_shape=[HEIGHT, WIDTH, 2])
    proto_d = tf.sparse.to_dense(tf.sparse.reorder(proto_sp))
    indices = tf.where(tf.logical_and(nnj_d, tf.not_equal(proto_d, 0)))
    values = tf.gather_nd(proto_d, indices)
    y, idx = tf.unique(values)
    return [y, tf.equal(y.get_shape()[0], 2)]

def bj_splitter(j, m, p, s, l, o, n, siid):
    """Body of while loop over neighbors"""
    nj = tf.gather(n, j)
    
    ids, pred = is_shared(nj, p, siid)
    
    def true_fn():
        nnew = tf.expand_dims(tf.concat([ids, nj], axis=0), axis=0)
        onew = tf.concat([o, nnew], axis=0)
        return [p, s, l, onew]
    def false_fn():
        nnew = tf.expand_dims(tf.concat([siid, nj], axis=0), axis=0)
        pnew = tf.concat([p, nnew], axis=0)
        lnew = tf.concat([l, nnew], axis=0)
        return [pnew, s, lnew, o]

    [pnew, snew, lnew, onew] = tf.cond(pred, true_fn, false_fn)
        
    return [tf.add(j, 1), m, pnew, snew, lnew, onew, n, siid]

def bi_splitter(i, m, p, s, l, o):
    """Body of while loop over current seed list"""
    si = tf.gather(s, i)
    
    # find neighbours not in protolist and only include cells originally clustered
    siid, siidx = tf.split(si, num_or_size_splits=[1,3], axis=0)
    _, pidx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    _, oidx = tf.split(o, num_or_size_splits=[2,3], axis=1)
    midx = tf.where(tf.greater(m, 0))
    
    n = neighbors(siidx, excl=tf.concat([pidx, oidx], axis=0), incl=midx)
    
    # loop over neighbors
    j0 = tf.constant(0)
    cj = lambda j, m, p, s, l, o, n, siid: tf.less(j, n.get_shape()[0])
    _, m, p, s, l, o, _, _ = tf.while_loop(
        cj, bj_splitter, loop_vars=[j0, m, p, s, l, o, n, siid],
        shape_invariants=[j0.get_shape(), m.get_shape(), tf.TensorShape([None, 4]),
                          s.get_shape(), tf.TensorShape([None, 5]), tf.TensorShape([None, 4]),
                          n.get_shape(), siid.get_shape()])
    
    return [tf.add(i, 1), m, p, s, l, o]

def finding_neighbors_splitter(m, p, s, o):
    """Finding neighbors recursively until current seed list is empty.
    m: image, p: protolist, s: seedlist, l: neighlist, n: neighbor, o: sharedlist"""
    slen = tf.shape(s)[0]
    
    # sort current seed list in descending order
    s = sorting_seedlist(s, slen, m)
    
    # loop over current seed list
    i0, l0 = tf.constant(0), tf.zeros([0,4], tf.int64)
    ci = lambda i, m, p, s, l, o: tf.less(i, slen)
    _, m, pnew, s, lnew, onew = tf.while_loop(
        ci, bi_splitter, loop_vars=[i0, m, p, s, l0, o],
        shape_invariants=[i0.get_shape(), m.get_shape(), 
                          tf.TensorShape([None, 4]), s.get_shape(),
                          tf.TensorShape([None, 4]), tf.TensorShape([None, 5])])
    
    # neighbor seed list becomes the new seed list
    snew = lnew
    
    # repeated until current seed list is empty
    def true_fn(): return finding_neighbors_splitter(m, pnew, snew, onew)
    def false_fn(): return [pnew, onew]
    [pnew, onew] = tf.cond(tf.not_equal(tf.size(snew), 0), true_fn, false_fn)
    return [pnew, onew]

### Shared cells

In [14]:
def bj_shared(j, p, l, n, siid):
    """Body of while loop over neighbors"""
    nj = tf.gather(n, j)
    nnew = tf.expand_dims(tf.concat([siid, nj], axis=0), axis=0)
    pnew = tf.concat([p, nnew], axis=0)
    lnew = tf.concat([l, nnew], axis=0)
    return [tf.add(j, 1), pnew, lnew, n, siid]

def bi_shared(i, m, p, s, l):
    """Body of while loop over current seed list"""
    si = tf.gather(s, i)
    
    # find neighbours not in protolist and only include cells originally clustered
    siid, siidx = tf.split(si, num_or_size_splits=[2,3], axis=0)
    _, pidx = tf.split(p, num_or_size_splits=[2,3], axis=1)
    midx = tf.where(m)
    
    n = neighbors(siidx, excl=pidx, incl=midx)
    
    # loop over neighbors
    j0 = tf.constant(0)
    cj = lambda j, p, l, n, siid: tf.less(j, n.get_shape()[0])
    _, p, l, _, _ = tf.while_loop(
        cj, bj_shared, loop_vars=[j0, p, l, n, siid],
        shape_invariants=[j0.get_shape(), tf.TensorShape([None, 5]), tf.TensorShape([None, 5]),
                          n.get_shape(), siid.get_shape()])
    
    return [tf.add(i, 1), m, p, s, l]

def expand_sharedlist(m, p, s):
    # loop over current seed list
    i0, l0 = tf.constant(0), tf.zeros([0,5], s.dtype)
    ci = lambda i, m, p, s, l: tf.less(i, s.get_shape()[0])
    _, _, pnew, s, lnew = tf.while_loop(
        ci, bi_shared, loop_vars=[i0, m, p, s, l0],
        shape_invariants=[i0.get_shape(), m.get_shape(), tf.TensorShape([None, 5]), 
                          s.get_shape(), tf.TensorShape([None, 5])])
    
    # neighbor seed list becomes the new seed list
    snew = lnew
    
    # repeated until current seed list is empty
    def true_fn(): return expand_sharedlist(m, pnew, snew)
    def false_fn(): return pnew
    slen = snew.get_shape()[0]
    pnew = tf.cond(tf.greater(slen, 0), true_fn, false_fn)
    return pnew

### Finalize cluster splitter

In [15]:
def finalize_splitter(m, p, o):
    pid, pidx = tf.split(p, num_or_size_splits=[1,3], axis=1)
    pval = tf.gather_nd(m, pidx)
    psp = tf.SparseTensor(pidx, tf.reshape(pid, [-1,]), dense_shape=[HEIGHT, WIDTH, 2])
    pd = tf.sparse.to_dense(tf.sparse.reorder(psp))
    pnew = tf.concat([tf.cast(pid, pval.dtype), tf.cast(pidx, pval.dtype), tf.expand_dims(pval, axis=1)], axis=1)
    
    y = tf.unique(tf.reshape(pid, [-1,]))[0]
#     print("unique pid", y)
    
    i0, com0, E0 = tf.constant(0), tf.zeros([0, 3], m.dtype), tf.zeros([0, 1], m.dtype)
    ci = lambda i, com, E: tf.less(i, y.get_shape()[0])
    def bi(i, com, E):
        yi = tf.gather(y, i)
        indices = tf.where(tf.equal(pd, yi))
        values = tf.expand_dims(tf.gather_nd(m, indices), axis=1)
        indices = tf.cast(indices, values.dtype)
        comi = tf.reduce_sum(tf.multiply(indices, values), axis=0)
        Ei = tf.reduce_sum(values)
        comi = tf.divide(comi, Ei)
        comi = tf.expand_dims(comi, axis=0)
        Ei = tf.reshape(Ei, [-1,1])
        return [tf.add(i, 1), tf.concat([com, comi], axis=0), tf.concat([E, Ei], axis=0)]
    _, com, E = tf.while_loop(
        ci, bi, loop_vars=[i0, com0, E0],
        shape_invariants=[i0.get_shape(), tf.TensorShape([None, 3]), tf.TensorShape([None, 1])])
#     print(com, E)
    
    j0, l0 = tf.constant(0), tf.zeros([0, 5], m.dtype)
    cj = lambda j, l: tf.less(j, o.get_shape()[0])
    def bj(j, l):
        oj = tf.gather(o, j)
        a, b, ojidx = tf.split(oj, num_or_size_splits=[1,1,3], axis=0)
        oval = tf.expand_dims(tf.gather_nd(m, ojidx), axis=0)
        ojidx = tf.cast(ojidx, com.dtype)
        amask = tf.equal(y, a)
        bmask = tf.equal(y, b)
        acom = tf.reshape(tf.boolean_mask(com, amask), [-1,])
        bcom = tf.reshape(tf.boolean_mask(com, bmask), [-1,])
        d1 = tf.sqrt(tf.reduce_sum(tf.math.squared_difference(ojidx, acom))) / moliereRadius
        d2 = tf.sqrt(tf.reduce_sum(tf.math.squared_difference(ojidx, bcom))) / moliereRadius
        # missing in units of !
        r = tf.exp(tf.subtract(d1, d2))
        E1 = tf.boolean_mask(E, amask)
        E2 = tf.boolean_mask(E, bmask)
        w1 = tf.reshape(tf.divide(E1, tf.add(E1, tf.multiply(r, E2))), [-1,])
        w2 = tf.subtract(1, w1)
        la = tf.expand_dims(tf.concat([tf.cast(a, com.dtype), ojidx, tf.multiply(w1, oval)], axis=0), axis=0)
        lb = tf.expand_dims(tf.concat([tf.cast(b, com.dtype), ojidx, tf.multiply(w2, oval)], axis=0), axis=0)
        return [tf.add(j, 1), tf.concat([l, la, lb], axis=0)]
    _, lnew = tf.while_loop(
        cj, bj, loop_vars=[j0, l0],
        shape_invariants=[j0.get_shape(), tf.TensorShape([None, 5])])

    pnew = tf.concat([pnew, lnew], axis=0)

    # sort in descending order in energy 
    indices = tf.reshape(tf.cast(tf.math.top_k(tf.reshape(E, [1, -1]), k=E.get_shape()[0])[1], o.dtype), [-1,])
#     print(y, indices)
    
    k0, p0 = tf.constant(0), tf.zeros([0, 5], pnew.dtype)
    ck = lambda k, p: tf.less(k, pnew.get_shape()[0])
    def bk(k, p):
        pk = tf.gather(pnew, k)
        pkid, pkidx = tf.split(pk, num_or_size_splits=[1,4], axis=0)
        yidx = tf.reshape(tf.where(tf.equal(y, tf.cast(pkid, y.dtype))), [-1,])
        pkid = tf.cast(tf.add(tf.where(tf.equal(indices, yidx)), 1), pkidx.dtype)
        pknew = tf.concat([pkid, tf.expand_dims(pkidx, axis=0)], axis=1)
        return [tf.add(k, 1), tf.concat([p, pknew], axis=0)]
    pnew = tf.while_loop(ck, bk, [k0, p0])[1]
    
    return pnew

### Cluster splitter

In [16]:
def cluster_splitter(parsed):    
    proto = parsed['proto']
    image = parsed['image']
#     image = tf.constant(image)
    
    # filter image with protolist
    proto_id, proto_idx = tf.split(proto, num_or_size_splits=[1,3], axis=1)
    proto_val = tf.gather_nd(image, proto_idx)
    image_sp = tf.SparseTensor(proto_idx, proto_val, dense_shape=[HEIGHT, WIDTH, 2])
    image_d = tf.sparse.to_dense(tf.sparse.reorder(image_sp))
    
    locmax = finding_local_maxima(image_d, proto)
    
    # create seed list from local maxima
    _, locmax_idx = tf.split(locmax, num_or_size_splits=[1,3], axis=1)
    proto_sp = tf.SparseTensor(proto_idx, tf.reshape(proto_id, [-1,]), dense_shape=[HEIGHT, WIDTH, 2])
    proto_d = tf.sparse.to_dense(tf.sparse.reorder(proto_sp))
    start = tf.add(tf.reduce_max(proto_id), 1)
    limit = start + locmax.get_shape()[0]
    locmax_id = tf.expand_dims(tf.range(start, limit, dtype=locmax.dtype), axis=1)
    seedlist = tf.concat([locmax_id, locmax_idx], axis=1)
    sharedlist = tf.zeros([0,5], tf.int64)
    
    [protolist, sharedlist] = finding_neighbors_splitter(image_d, seedlist, seedlist, sharedlist)
    
    # originally clustered cells not in protolist
    protolist_id, protolist_idx = tf.split(protolist, num_or_size_splits=[1,3], axis=1)
    protolist_sp = tf.SparseTensor(protolist_idx, tf.reshape(protolist_id, [-1,]), dense_shape=[HEIGHT, WIDTH, 2])
    protolist_d = tf.sparse.to_dense(tf.sparse.reorder(protolist_sp))
    mask = tf.logical_and(tf.cast(proto_d, tf.bool), tf.logical_not(tf.cast(protolist_d, tf.bool)))
    
    sharedlist = expand_sharedlist(mask, sharedlist, sharedlist)
    
    # add parent clusters without a local maximum
    mask = tf.logical_and(tf.cast(proto_d, tf.bool), tf.cast(protolist_d, tf.bool))
    proto_y = tf.expand_dims(tf.unique(tf.reshape(proto_id, [-1,]))[0], axis=0)
    proto_masked_y = tf.expand_dims(tf.unique(tf.boolean_mask(proto_d, mask))[0], axis=0)
    other = tf.sparse.to_dense(tf.sets.difference(proto_y, proto_masked_y))

    other = tf.reshape(other, [-1,1])
    
    if tf.not_equal(tf.size(other), 0):
        i = tf.constant(0)
        c = lambda i, p: tf.less(i, tf.shape(other)[0])
        def b(i, p): 
            otheri = tf.gather(other, i)
            p = tf.where(tf.equal(proto_d, otheri), proto_d, p) 
            return [tf.add(i, 1), p]
        protolist_d = tf.while_loop(c, b, [i, protolist_d])[1]
        pidx = tf.where(protolist_d)
        pid = tf.expand_dims(tf.gather_nd(protolist_d, pidx), axis=1)
        protolist = tf.concat([pid, pidx], axis=1)
    
    cluster = finalize_splitter(image_d, protolist, sharedlist)
        
    parsed['cluster'] = cluster
    
    return parsed

## Discriminating variables

In [17]:
def scalar_features(parsed):
    cluster = parsed['cluster']
    
    cid, cidx, cval = tf.split(cluster, num_or_size_splits=[1,3,1], axis=1)    
    y = tf.unique(tf.reshape(cid, [-1,]))[0]
#     print("unique pid", y)
    
    i0 = tf.constant(0)
    z0 = tf.zeros([0, 8], cluster.dtype)
    c = lambda i, z: tf.less(i, tf.size(y))
    def bi(i, z):
        yi = tf.gather(y, i)
        cid_indices = tf.where(tf.equal(tf.reshape(cid, [-1,]), yi))
        values = tf.gather_nd(cval, cid_indices)
        indices = tf.reshape(tf.gather(cidx, cid_indices), [-1,3])
        cij, ck = tf.split(indices, num_or_size_splits=[2,1], axis=1)
        com = tf.reduce_sum(tf.multiply(cij, values), axis=0, keepdims=True)/tf.reduce_sum(values)
        
        S_indices = tf.where(tf.equal(tf.reshape(ck, [-1,]), 0))
        S_values = tf.gather_nd(values, S_indices)
        S_cij = tf.reshape(tf.gather(cij, S_indices), [-1,2])
        S_sum = tf.reduce_sum(S_values, keepdims=True)
        S_max = tf.cond(tf.greater(S_sum, 0.), lambda: tf.reduce_max(S_values), lambda: tf.constant([[0.]]))
        S_hot = tf.cond(tf.greater(S_sum, 0.), lambda: S_max/S_sum, lambda: tf.constant([[0.]]))
        S_com = tf.reduce_sum(tf.multiply(S_cij, S_values), axis=0)/tf.reduce_sum(S_values)
        S_rad = tf.sqrt(tf.reduce_sum(tf.pow(S_cij-S_com, 2), axis=1, keepdims=True))
        S_rad_mean = tf.cond(tf.greater(S_sum, 0.), 
                             lambda: tf.reduce_sum(tf.multiply(S_rad, S_values), axis=0, keepdims=True)/tf.reduce_sum(S_values),
                             lambda: tf.constant([[0.]]))
        
        C_indices = tf.where(tf.equal(tf.reshape(ck, [-1,]), 1))
        C_values = tf.gather_nd(values, C_indices)
        C_cij = tf.reshape(tf.gather(cij, C_indices), [-1,2])
        C_sum = tf.reduce_sum(C_values, keepdims=True)
        C_max = tf.cond(tf.greater(C_sum, 0.), lambda: tf.reduce_max(C_values), lambda: tf.constant([[0.]]))
        C_hot = tf.cond(tf.greater(C_sum, 0.), lambda: C_max/C_sum, lambda: tf.constant([[0.]]))
        C_com = tf.reduce_sum(tf.multiply(C_cij, C_values), axis=0)/tf.reduce_sum(C_values)
        C_rad = tf.sqrt(tf.reduce_sum(tf.pow(C_cij-C_com, 2), axis=1, keepdims=True))
        C_rad_mean = tf.cond(tf.greater(C_sum, 0.),
                             lambda: tf.reduce_sum(tf.multiply(C_rad, C_values), axis=0, keepdims=True)/tf.reduce_sum(C_values),
                             lambda: tf.constant([[0.]]))
     
        zi = tf.concat([com, S_sum, S_rad_mean, S_hot, C_sum, C_rad_mean, C_hot], axis=1)
        
        return [tf.add(i, 1), tf.concat([z, zi], axis=0)]
    _, z = tf.while_loop(c, bi,
                         loop_vars=[i0, z0],
                         shape_invariants=[i0.get_shape(), tf.TensorShape([None, 8])])
    parsed['scalar'] = z
    return parsed